# Section 3: RCTs

Today we are going to go over RCTs somewhat interactively in R, with the hope of building familiarity with the type of analysis you might do on your final projects. As in class, we are going to go over the _Cash or Condition_ paper by Baird et al (2011). Recall that the paper studies a randomized control trial in which teenage girls in Malawi were offered a cash transfer that was either conditional on their school attendance or unconditional or were offered no cash transfer. The outcome we will mostly be looking at today is how each type of transfer `T2a` and `T2b` affected teen pregnancy rates `ever_pregnant`. 

In [ ]:
# load packages we will need
pacman::p_load(tidyverse, haven, lfe, modelsummary)
# load data
df <- read_stata('CorC_Public_Data_FINAL_PubH6445.dta') 
# get just panel data from round 3
df3 <- df %>% subset(panel == T & round == 3)

## What is an RCT?
The main challenge of identifying the causal impact of a policy or program is constructing a valid counterfactual for what would have happened to treated individuals had they not been treated. Of course, we never observe the same individual being both treated and untreated at a given time. So the next best thing we can do is compare groups of people that were statistically identical before the treatment after some of them get the treatment and others don't. We can do this by randomizing who gets treated and who doesn't. If treatment is random, we can be almost positive that the _only_ difference between the treatment and control group is the effect of the treatment, as long as our sample is large enough. 

When feasible, randomization is the most rigorous approach to construct a treatment and a control group from among an eligible population (see Duflo, 2006; Banerjee and Duflo, 2009).

How do we analyze an RCT If we want to know the **average treatment effect (ATE)** of a program, then all we need to do is compare the average value of our outcome in the treatment group to its average value in the control group.

$$ ATE = \bar Y_T - \bar Y_C$$

We could also accomplish this (and get standard errors) by doing this in regression form,

$$ Y_i = \alpha + \beta Treat_i + u_i $$
where $\beta$ is the ATE. 

Of course, in our Malawi case, we have two mutually-exclusive treatments, but this doesn't complicate things much. We can still estimate the ATEs for each of the conditional (CCT) and unconditional (UCT) cash transfers

$$ EverPregnant_i = \alpha + \beta_1 CCT_i + \beta_2 UCT_i + u_i $$

Here $\beta_1$ is the average treatment effect of receiving a conditional cash transfer and $\beta_2$ is the average treatment effect of receiving an unconditional cash transfer on teen pregnancy.

1. Compute $\beta_1$ and $\beta_2$ without a regression by comparing means across groups.
2. Estimate $\beta_1$ and $\beta_2$ from a regression. 



In [ ]:
# First check the names of what we're working with
names(df3)

Note `C2` is the indicator for the control group

In [ ]:
# Take means for each group
Y_CCT <- df3[df3$T2a==1,]$ever_pregnant %>% mean() 
Y_UCT <- df3[df3$T2b==1,]$ever_pregnant %>% mean() 
Y_C <- df3[df3$C2==1,]$ever_pregnant %>% mean()

# B1 and B2
print(paste0('Effect of CCT: ',Y_CCT-Y_C,' Effect of UCT: ', Y_UCT-Y_C))

In [ ]:
# Regression version
felm(ever_pregnant~T2a+T2b, data= df3) %>% summary()

The coefficients on `T2a` and `T2b` are exactly the same as the raw differences we calculated. It seems like CCTs caused a small (but statistically insignificant) increase in pregnancy while the UCTs caused a large drop in pregnancy, but this falls just short of statistical significance. In addition, the intercept tells us what the mean pregnancy rate in the control group was.

In [ ]:
Y_C

## A Few Practical Concerns

Most household surveys come with sampling weights. These weights assign different importance to different observations (inversely) based on the probability that they were included in the sample. This is because it's not usually feasible to survey a group of people that's exactly representative of the population of interest. But fortunately, surveys come with weights that we can apply in order to make the sample representative of the broader population. We can implement this in R by specifying the column containing our sampling weights `wgt` in the `weights` argument in `felm`. 

We also want to make sure we're using the right standard errors to conduct statistical inference. The default standard errors we get when we run `felm` or some other regression package are almost never the ones we actually want, because they make unrealistic assumptions about the distribution of the error term. When it comes to RCTs, best practice is to cluster standard errors at the level at which treatment was assigned. (This essentially allows the error term to be arbitrarily correlated within clusters. For example, we might expect unobserved factors affecting pregnancy to more highly correlated among girls from the same village. This generally produces more conservative estimates than unadjusted standard errors). In this case, treatment was assigned at the community level so we can use the variable `eaid` to cluster. We can simply specify this with the `cluster` argument in `felm` (for now at least). 

3. Re-run your regression applying the appropriate sampling weights. Then also cluster your standard errors at the village level. How does this affect the standard errors? How does this affect the coefficients?

In [ ]:
# Weighting 
felm(ever_pregnant~T2a+T2b, data= df3, weights=df3$wgt) %>% summary()


Weighting changes our effects sligtly. We see both coefficients decrease and the coefficient on UCTs becomes highly significant now. Again, the two regressions are telling us slightly different things. The original regression was telling us the average effects on girls in the sample and this is telling us what the effects would be in the population, accounting for the fact that our sample doesn't exactly match the population in terms of demographics. 

In [ ]:
# Weighting and clustering
felm(ever_pregnant~T2a+T2b, data= df3, weights=df3$wgt, cluster = df3$eaid) %>% summary()

Clustering doesn't change our coefficients at all, we're only changing the way in which we interpret them through the standard errors. The standard errors increase a little bit, which is what you would tend to expect from clustering. It's always good to be a little conservative with standard errors. 

## Tests and Adjustments

Now that we've got the mechanics down, should we believe these estimates we've just computed? The key assumption is that the randomization did in fact result in two statistically identicial groups. Unfortunately, we can't test this assumption directly because there are many variables we can't observe. But as a smell test, we can at least test whether observable attributes (that are either measured before the treatment or are fixed) are statistically different between the two groups. 

4. Statistically test whether the baseline variables `age_R1`, `highest_grade_bl`, `asset_index_bl` and `never_had_sex_bl` vary across groups. First compare CCT to control and then UCT to control. 

In [ ]:
# We want to compare CCT to Control so drop the UCT group and conduct a t-test by T2a
bal_CCT <- df3 %>% subset(T2b==0) %>% select(T2a, age_R1, highest_grade_bl,asset_index_bl,never_had_sex_bl, wgt)
"CCT vs. Control: p-values"
"Age"
t.test(bal_CCT$age_R1~bal_CCT$T2a)$p.value
"Highest Grade"
t.test(bal_CCT$highest_grade_bl~bal_CCT$T2a)$p.value
"Asset Index"
t.test(bal_CCT$asset_index_bl~bal_CCT$T2a)$p.value
"Never Had Sex"
t.test(bal_CCT$never_had_sex_bl~bal_CCT$T2a)$p.value


bal_UCT <- df3 %>% subset(T2a==0) %>% select(T2b, age_R1, highest_grade_bl,asset_index_bl,never_had_sex_bl)
"UCT vs. Control: p-values"
"Age"
t.test(bal_UCT$age_R1~bal_UCT$T2b)$p.value
"Highest Grade"
t.test(bal_UCT$highest_grade_bl~bal_UCT$T2b)$p.value
"Asset Index"
t.test(bal_UCT$asset_index_bl~bal_UCT$T2b)$p.value
"Never Had Sex"
t.test(bal_UCT$never_had_sex_bl~bal_UCT$T2b)$p.value



# # The modelsummary package has a nice way to make tables automatically if you're doing this for many variables (uncomment to run)
# pacman::p_load(modelsummary)

# # Test CCT vs. Control
# datasummary_balance(~T2a, bal_CCT ,output= 'jupyter', fmt =2, dinm_statistic = 'p.value')
# ## Test UCT vs. Control
# datasummary_balance(~T2b, bal_UCT ,output= 'jupyter', fmt =2, dinm_statistic = 'p.value')

Okay, so we see some statistically significant differences across treatment and control. (My hunch is that it's because we didn't weight the t-tests but don't worry about that here). Do we necessarily have to worry about these differences though? Not really. As long we believe the randomization was done properly we can always control for them in order to hold these attributes that happen to differ across groups fixed. 

Another advantage of adding (pre-determined) controls is that they increase the precision of our treatment effect estimates by soaking up a lot of the additional variation that was not explained by the model without controls. 

5. Add the covariates you used above to the regression, but use the dummy variables `_Iage_R1_14-20` instead of `age_R1` and also control for `stratum1` and `stratum2`.
   
   a. Are these additional variables statistically significant? Is this surprising or not?
   
   b. How do the treatment effects you estimate compare to the ones you estimated without controls. Does this suggest that the sample imbalances you found were affecting our results or not?
   
   c. What happens to the precision of our estimates when we add controls?
   

In [ ]:
names(df3) <- gsub('^_','',names(df3))
felm(ever_pregnant~T2a+T2b+
     highest_grade_bl+
     asset_index_bl+
     never_had_sex_bl+
     Iage_R1_14 +
     Iage_R1_15 +
     Iage_R1_16 +
     Iage_R1_17 +
     Iage_R1_18 +
     Iage_R1_19 +
     Iage_R1_20 + 
     stratum1 + stratum2|0|0|eaid, data = df3, weights = df3$wgt) %>% 
     summary()

First, it shouldn't be surprising that these other variables are significant. Of course things like age are going to be correlated with pregnancy both within the treatment group and within the control group. The UCT treatment makes pregnancy less common, but those who do get pregnant are still likely to be older. We don't really care about these coefficients though because we're interested in treatment effects. 

The treatment effects only change a little bit, which is somewhat reassuring, as it suggests that not controlling for these factors wasn't biasing our estimates too much. However, our standard errors go down. This is because standard errors are higher when there is a lot of variation in the outcome that the model fails to explain. Adding covariates helps soak up this variation, which lowers our standard errors and gives us more precise results.

So again, the two main reasons to add controls to an RCT -- even though they are _not_ necessary for an RCT to be conceptually valid -- are that they control for potential imbalances in observables and increase the precision of our estimates. 

**Warning:** You should only use variables that are either measured prior to treatment or are fixed characteristics. Under no circumstances should you include a variable that can be affected by treatment on the right-hand side.

For example, if we had controlled for school attendance (post-treatment), we would get a biased estimate. The regression would tell us the effect of UCTs and RCTs _holding school attendance fixed_. But we don't want to hold this fixed because much of the effect of the cash transfers operates through school attendance, so we end up throwing out this part of the effect and are left with a meaningless estimate that lacks a causal interpretation. We therefore would call school attendance a **bad control**.

Note that we've only been using the third round of data so far. The nice thing about an RCT is that we don't even need pre-period data because the only different between groups post-treatment is the effect of the treatment. But we still can use pre-period data, again both as a robustness check and to increase precision, by running a diff-in-diff. Randomization ensures that trends in pregnancy (like everything else) are the same across groups, so we get parallel trends for free. 

6. Estimate treatment effects using the full dataset `df`. Assume the "post" period is round 3. How have the coefficients changed? Is this what you expected?

In [ ]:
df$post <- df$round==3
df$r <- df$round %>% as.factor()

In [ ]:
felm(ever_pregnant~T2a*post+T2b*post|0|0|eaid, data = df, weights = df$wgt) %>%  summary()

(note that the `*` in the regression formula is an interaction term. So `T2a*post` is equal to including $T2a+post+T2a\times post$ in the regression.)
The bottom two coefficients are the difference-in-difference coefficients that we care about. Their magnitudes are highly similar to the RCT using only a single round of data, but this is another useful robustness check when we have pre-period data. 

## ATE vs ITT
Finally, let's consider what would happen if not everybody actually took the treatment. We wouldn't expect people to turn down free money, but with other treatments this is much more common. Turning down the treatment is definitely not random and is correlated with all types of omitted variables, so we can't just compare people who were offered the treatment and chose to take it with those who weren't offered 

Sometimes we don't have perfect **compliance** among the treatment and control groups: some observations in the treatment group do not take the treatment; and some observations in the control group find a way to take the treatment.  This decision to ``not comply" with the research design is almost certainly correlated with other unobservable characteristics. In the case of Malawi, think of girls who for some reason didn't show up to claim their cash transfer. They are probably very different in lots of ways from those who did show up to claim free money. In this case, if we decide to compare those in the treatment group who actually got the transfer, to those in the control group who did not, we are no longer comparing the randomly assigned groups, and we won't get a causal estimate. 

We call people who take the treatment if and only if they are put in the treatment group **compliers**. These are the people that are going to give us the variation we can use to estimate the causal effects. In contrast, people who take the treatment regardless of whether we assign the treatment (**always takers**) or who never take the treatment even when we offer it (**never takers**), are never going to be affected by the RCT. 

|                    | Assigned to Treatment | Assigned to Control |
|--------------------|-----------------------|---------------------|
| Treated            | T Complier            | Always-Taker        |
| Not Treated        | Never-Taker           | C Complier          |


What do we do in this case?  We acknowledge that we don't have perfect compliance, but we continue to compare those who were _assigned_ to the treatment group, to those who were _assigned_ to the control group. So if you are in the treatment group but didn't comply, you will still be considered treated, and if you are in the control group and didn't comply you will still be considered control. In other words, we are estimating the effect of being _offered_ a treatment, which is still very much random regardless of what you end up doing with it. We therefore call this the estimator **Intention to Treat (ITT)**. We estimate the ITT using the same specification as the ATE - the only difference is that we acknowledge that our treatment assignment was not perfectly successful, so we no longer estimate the ATE.

We can always estimate the ITT with an RCT. The estimate we recover will reflect the fact that some people in the treatment group received the treatment, but the total effect will be attenuated by the fact that not everyone received the treatment (and that perhaps some control observations did). 
If the treatment only affects the outcome through its effect in varying a characteristic of interest (receiving a cash transfer), the ITT tells us the effect of that characteristic on the outcome, diluted by the fact that not every treated observation complied with the treatment (e.g.,actually got the money).

The ITT estimate thus gives us the impact of the treatment among compliers, weighted by the share of compliers. Observe that if everyone complies with the treatment, this is the same as the ATE.

We can "undilute" the ITT and compute the **Average Treatment on the Treated (ATT)** (sometimes you might hear this called ToT) if we're willing to believe that the only way being offered the treatment affects your outcome is through actually taking the treatment. So if you're offered a cash transfer and don't accept it for whatever reason, we're assuming that your outcome is going to be the same as if you were never offered the cash transfer at all. In this case, we can compute the ATT by dividing the ITT by the share of compliers

$$ ATT = \frac{ITT}{\text{\% of compliers}} $$

Dividing by the share of compliers is essentially undoing the dilution of the ITT by people who are not induced to change by the treatment. To see this, let's just think about UCTs for now.  Assume that the effect of _receiving_ a cash transfer on pregnancy rates (at least for compliers) is $\beta$  So $\beta$ is the ATT. Suppose 70\% of people who were offered the transfer actually took it. Then if we estimated the ITT, we'd get:

$$ITT =  .7\times\beta + .3\times0 = .7\beta $$

This is because the 70\% of people who took the transfer get the effet $\beta$, but nothing happens to the other 30\% of people. So dividing by the complier share

$$ ATT = \frac{.7\beta}{.7} = \beta $$

gives us the ATT. 

Note that the ATT is not necessarily the same as the ATE, which is the average effect of the program on the whole population. We are only estimating the effect of the program on compliers, but maybe people who turned down the cash would have been more or less affected by it. The ATT is still highly relevant for policy though: it's the effect of the program for people who are actually impacted by it.  


